# Table of Contents
* [Prepare the environment](#Prepare-the-environment)
* [Parameters for download sources](#Parameters-for-download-sources)
* [Define functions](#Define-functions)
* [Download of data](#Download-of-data)
* [Adjustment of BELGIAN data](#Adjustment-of-BELGIAN-data)
	* [Import data](#Import-data)
	* [Translate and adjust columns](#Translate-and-adjust-columns)
	* [Generate information concerning Type and CHP](#Generate-information-concerning-Type-and-CHP)
	* [Translate and adjust Technology types](#Translate-and-adjust-Technology-types)
	* [Translate and adjust Fuel types](#Translate-and-adjust-Fuel-types)
	* [Generate information concerning Technology](#Generate-information-concerning-Technology)
* [Adjustment of DUTCH data](#Adjustment-of-DUTCH-data)
	* [Import and merge data](#Import-and-merge-data)
	* [Translate and adjust columns](#Translate-and-adjust-columns)
	* [Translate and adjust Fuel types](#Translate-and-adjust-Fuel-types)
	* [Adjust Capacity](#Adjust-Capacity)
* [Adjustment of ITALIAN data](#Adjustment-of-ITALIAN-data)
	* [Import data](#Import-data)
	* [Translate and adjust columns](#Translate-and-adjust-columns)
	* [Translate and adjust Fuel types](#Translate-and-adjust-Fuel-types)
* [Adjustment of FRENCH data](#Adjustment-of-FRENCH-data)
	* [Import data](#Import-data)
	* [Translate and adjust Columns](#Translate-and-adjust-Columns)
	* [Translate and adjust Fuel types](#Translate-and-adjust-Fuel-types)
	* [Generate Technology types](#Generate-Technology-types)
* [Adjustment of FINNISH data](#Adjustment-of-FINNISH-data)
	* [Import data](#Import-data)
	* [Translate and adjust columns](#Translate-and-adjust-columns)
	* [Translate and adjust Fuel types](#Translate-and-adjust-Fuel-types)
	* [Generate Technology types](#Generate-Technology-types)
	* [Translate and adjust Types](#Translate-and-adjust-Types)
* [Adjustment of POLISH Data](#Adjustment-of-POLISH-Data)
	* [Import data](#Import-data)
	* [Translate and adjust Columns](#Translate-and-adjust-Columns)
	* [Translate and adjust Fuel types](#Translate-and-adjust-Fuel-types)
	* [Generate Technology types](#Generate-Technology-types)
* [Adjustment of SPANISH data](#Adjustment-of-SPANISH-data)
	* [Import data](#Import-data)
	* [Translate and adjust Columns](#Translate-and-adjust-Columns)
	* [Translate and adjust Fuel types](#Translate-and-adjust-Fuel-types)
	* [Translate and adjust Technology types](#Translate-and-adjust-Technology-types)
	* [Generate and adjust Fuel types](#Generate-and-adjust-Fuel-types)
* [Adjustment of BRITISH data](#Adjustment-of-BRITISH-data)
	* [Import data](#Import-data)
* [Translate and adjust Columns](#Translate-and-adjust-Columns)
	* [Generate and adjust Technology types](#Generate-and-adjust-Technology-types)
	* [Translate and adjust Fuel types](#Translate-and-adjust-Fuel-types)
* [Create output-files](#Create-output-files)
* [Documenting the data package (meta data)](#Documenting-the-data-package-%28meta-data%29)
* [Write results to file](#Write-results-to-file)
* [STEINBRUCH [not relevant]](#STEINBRUCH-[not-relevant])


# Prepare the environment

Import of every module and function needed to process the data and creation of the target folders.

In [ ]:
import requests
import os
import urllib.request
import zipfile
import csv
import pandas as pd
import numpy as np
import posixpath
import urllib.parse
import datetime  
import re
import os.path
import yaml  # http://pyyaml.org/, pip install pyyaml, conda install pyyaml
import json
import subprocess
from bokeh.charts import Scatter, show
from bokeh.io import output_notebook
output_notebook()
%matplotlib inline
import logging
logger = logging.getLogger('notebook')
logger.setLevel('INFO')
nb_root_logger = logging.getLogger()
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                              datefmt='%d %b %Y %H:%M:%S')
nb_root_logger.handlers[0].setFormatter(formatter)

#create download and output folder if they do not exist
os.makedirs('data_downloaded', exist_ok=True)
os.makedirs('data_processed', exist_ok=True)
os.makedirs('data_final', exist_ok=True)

# Parameters for download sources

Input of relevant meta data for every country and data source to facilitate the download loop. 

In [ ]:
conf = """
    BE: 
        Elia: 
            url_template: http://publications.elia.be/upload/ProductionParkOverview.xls?TS=20120416193815
            filename: ProductionParkOverview
            filetype: xls
            sheetname: 'ProductionParkOverview'
            skiprows: 1
    NL: 
        Tennet_Q12015: 
            url_template: http://www.tennet.org/english/operational_management/export_data.aspx?exporttype=installedcapacity&format=csv&quarter=2015-1&submit=3
            filename: export_Q12015
            filetype: csv
            sep: ','
            skiprows: 0
            decimal: '.'
            encoding: 'utf-8'
        Tennet_Q22015: 
            url_template: http://www.tennet.org/english/operational_management/export_data.aspx?exporttype=installedcapacity&format=csv&quarter=2015-2&submit=3
            filename: export_Q22015
            filetype: csv
            sep: ','
            skiprows: 0
            decimal: '.'
            encoding: 'utf-8'
        Tennet_Q32015: 
            url_template: http://www.tennet.org/english/operational_management/export_data.aspx?exporttype=installedcapacity&format=csv&quarter=2015-3&submit=3
            filename: export_Q32015
            filetype: csv
            sep: ','
            skiprows: 0
            decimal: '.'
            encoding: 'utf-8'
        Tennet_Q42015: 
            url_template: http://www.tennet.org/english/operational_management/export_data.aspx?exporttype=installedcapacity&format=csv&quarter=2015-4&submit=3
            filename: export_Q42015
            filetype: csv
            sep: ','
            skiprows: 0
            decimal: '.'
            encoding: 'utf-8'
    IT:
        Terna: 
            url_template: http://download.terna.it/terna/0000/0216/16.XLSX
            filename: 16
            filetype: xls
            sheetname: 'UPR PmaxOver 100MW'
            skiprows: 0
    
    FR: 
        RTE: 
            url_template: http://clients.rte-france.com/servlets/CodesEICServlet
            filename: Centrales_production_reference
            filetype: zip
            sep: '\t'
            skiprows: 2
            decimal: ','
            encoding: 'cp1252'
            
    ES: 
        SEDE: 
            url_template: http://www6.mityc.es/aplicaciones/electra/ElectraExp.csv.zip
            filename: ElectraExp
            filetype: zip
            sep: ';'
            skiprows: 0
            decimal: ','
            encoding: 'utf-8'

    FI: 
        EnergyAuthority: 
            url_template: http://www.energiavirasto.fi/documents/10191/0/Energiaviraston+Voimalaitosrekisteri+040316.xlsx
            filename: Energiaviraston+Voimalaitosrekisteri+040316
            filetype: xlsx
            sheetname: 'English'
            skiprows: 1
            
#    DK: 
#        EnerginetDK: 
#            url_template: https://www.energinet.dk/SiteCollectionDocuments/Engelske%20dokumenter/El/Energinet%20dk%27s%20assumptions%20for%20analysis%202014-2035,%20September%202014.xlsm
#            filename: Energinet dk's assumptions for analysis 2014-2035, September 2014
#            filetype: xlsm            
  
    PL: 
        GPI: 
            url_template: http://gpi.tge.pl/en/wykaz-jednostek?p_p_id=powerunits_WAR_powerunitsportlet&p_p_lifecycle=2&p_p_state=normal&p_p_mode=view&p_p_cacheability=cacheLevelPage&p_p_col_id=column-1&p_p_col_count=1
            filename: units_list
            filetype: csv            
            sep: ';'
            skiprows: 0
            decimal: '.'
            encoding: 'utf-8'

    UK: 
        GOV: 
            url_template: https://www.gov.uk/government/uploads/system/uploads/attachment_data/file/446457/dukes5_10.xls
            filename: dukes5_10
            filetype: xls
            sheetname: 'Database'
            skiprows: 3
  
  """

conf = yaml.load(conf)

# Define functions

Definition of the download process and adjustment of the file name according to the acutal date.

In [ ]:
def downloadandcache(url,filename,filetype):
    """This function downloads a file into a folder called 
    data-downloaded and returns the local filepath."""
    path = urllib.parse.urlsplit(url).path
#   filename = posixpath.basename(path)
    filename = str(filename)
    filetype = filetype
    now = datetime.datetime.now()
    datestring = ""
    datestring = str(now.year)+"-"+str(now.month)+"-"+str(now.day)
    filepath = "data_downloaded/"+datestring+"-"+filename+"."+filetype
    filepath_original_data = "data_original/"+filename+"."+filetype
    
    #check if file exists, otherwise download it
    if os.path.exists(filepath) == False:
        print("Downloading file", filename+"."+filetype)
        urllib.request.urlretrieve(url, filepath)
        urllib.request.urlretrieve(url, filepath_original_data)
    else:
        print("Using local file from", filepath)
    filepath = './'+filepath
    return filepath

def importdata(country,tso):
    now = datetime.datetime.now()
    datestring = ""
    datestring = str(now.year)+"-"+str(now.month)+"-"+str(now.day)
    data_import=pd.DataFrame()
    param = conf[country][tso]
    filepath = "data_downloaded/"+datestring+"-"+str(param['filename'])+"."+param['filetype']
    if param['filetype'] == 'csv':
        data_import = pd.read_csv(filepath,
                              sep=param['sep'],
                              skiprows=param['skiprows'],
                              decimal=param['decimal'],   
                              encoding=param['encoding'])
    else: 
        data_import = pd.read_excel(filepath,
                                    sheetname=param['sheetname'],
                                    skiprows=param['skiprows'])
    
    data_import['Country'] = str(country)
    data_import['Source'] = str(tso)
    
    return data_import

# Download of data

Start of the download loop. Consideration of two exceptions due to file types (ZIP-file).

In [ ]:
for country, tso in conf.items():
    for tso, param in tso.items():
        print(param['url_template'])
#        make_url(param['url_template'],param['filetype'])
        downloadandcache(param['url_template'],param['filename'],param['filetype'])

# Special case FR (RTE): ZIP-file with corrupted xls-file is provided, which needs to be renamed to csv 
    if country=='FR':
        now = datetime.datetime.now()
        datestring = str(now.year)+"-"+str(now.month)+"-"+str(now.day)
        filepath = "data_downloaded/"+datestring+"-"+param['filename']
        
        with zipfile.ZipFile(filepath+".zip","r") as O:
            O.extractall("data_downloaded/")
        if os.path.exists(filepath+".csv") == False:
            os.rename("data_downloaded/"+param['filename']+".xls",filepath+".csv")
        # change filetype from zip to csv
        conf[country][tso]['filetype'] = "csv"

# Special case ES (SEDE): ZIP-file with csv file 
    if country=='ES':
        now = datetime.datetime.now()
        datestring = str(now.year)+"-"+str(now.month)+"-"+str(now.day)
        filepath = "data_downloaded/"+datestring+"-"+param['filename']
        
        with zipfile.ZipFile(filepath+".zip","r") as O:
            O.extractall("data_downloaded/")
        if os.path.exists(filepath+".csv") == False:
            os.rename("data_downloaded/"+param['filename']+".csv",filepath+".csv")
        # change filetype from zip to csv
        conf[country][tso]['filetype'] = "csv"
#        print(conf)

# Adjustment of BELGIAN data

## Import data

In [ ]:
# Check if input works
data_BE = importdata('BE','Elia')

## Translate and adjust columns

Overall adjustment of all columns within the dataframe. 
Translation, addition, deletion, sorting of columns as well as adjustment of the column entries' types.

In [ ]:
# Drop columns not needed anymore
colsToDrop = ['Unnamed: 0', 
              'Unnamed: 2', 
              'Unnamed: 4', 
              'Unnamed: 6', 
              'Unnamed: 7', 
              'Fuel for publication',
              'Unnamed: 11']
data_BE = data_BE.drop(colsToDrop, axis=1)

# Translate columns
dict_columns_BE = {'ARP':'Company',
                   'Generation plant':'Name',
                   'Plant Type':'Technology',
                   'Technical Nominal Power (MW)':'Capacity',
                   'Remarks':'Comment',
                   'Fuel':'Fuel',
                   'Country':'Country',
                   'Source':'Source'}
data_BE.rename(columns=dict_columns_BE, inplace=True)

# Check if all columns have been translated
for columnnames in data_BE.columns:
    #print(columnnames)
    if not columnnames in dict_columns_BE.values():
        logger.error("Untranslated column: "+ columnnames)

# Add needed columns without data
data_BE['Street', 
        'Postcode', 
        'City',
        'CHP',
        'Type'] = 'NaN'

# Sort columns
columns_sorted_BE = ['Company',
                     'Name',
                     'Street',
                     'Postcode',
                     'City',
                     'Country',
                     'Capacity',
                     'Fuel',
                     'Technology',
                     'Type',
                     'CHP',
                     'Comment',
                     'Source']
data_BE = data_BE.reindex(columns=columns_sorted_BE)

# Drop rows without capacity entries, so that row with "Unit connected to Distribution Grid" is dropped
data_BE = data_BE.dropna(subset=['Capacity'])

#Adjust types of entries in all columns
data_BE.Company = data_BE.Company.astype(str)
data_BE.Name = data_BE.Name.astype(str)
data_BE.Street = data_BE.Street.astype(str)
data_BE.Postcode = data_BE.Postcode.astype(str)
data_BE.City = data_BE.City.astype(str)
data_BE.Country = data_BE.Country.astype(str)
data_BE.Capacity = data_BE.Capacity.astype(float)
data_BE.Fuel = data_BE.Fuel.astype(str)
data_BE.Technology = data_BE.Technology.astype(str)
data_BE.Type = data_BE.Type.astype(str)
data_BE.CHP = data_BE.CHP.astype(str)
data_BE.Comment = data_BE.Comment.astype(str)
data_BE.Source = data_BE.Source.astype(str)

## Generate information concerning Type and CHP

Generation of entries for the columns "Type" and "CHP" according to information given in the columns "Technology" and "Type"

In [ ]:
# Generate entries in column "type" according to technology "WKK"
data_BE['Type'][data_BE['Technology'] == 'WKK'] = 'CHP'

# Generate entries in column "CHP" according to column "type"
data_BE['CHP'][data_BE['Type'] == 'CHP'] = 'Yes'

## Translate and adjust Technology types

TODO: workaround for 'WKK' - Extract technology from name

Overall translation of all technology types mentioned in the column "Technology" and subsequent translation check.

In [ ]:
# Translate technologies
dict_technology_BE = {'BG':'GT',
                      'CL': 'UNKNOWN',
                      'WKK': 'UNKNOWN',
                      'CCGT': 'CC',
                      'D':'UNKNOWN',
                      'HU':'hydro_plant',
                      'IS':'UNKNOWN',
                      'NU':'ST',
                      'TJ':'UNKNOWN',
                      'WT':'wind_turbine',
                     ' ':'UNKNOWN'}
data_BE["Technology"].replace(dict_technology_BE, inplace=True)
data_BE["Technology"].unique()

# Check if all technologies have been translated
for technology in data_BE["Technology"].unique():
    if (not technology in dict_technology_BE.values()) & (str(technology) != "nan"):
        logger.error("Untranslated technology: " + str(technology))

## Translate and adjust Fuel types

Overall translation of all fuel types mentioned in the column "Fuel" and subsequent translation check. Deletion of rows containing "wind" as fuel type.

In [ ]:
# Translate fuel types
dict_fuels_BE = {'BIO':'biomass',
                 'BF':'gas',
                 'CL':'lignite',
                 'CP':'coal',
                 'CG':'gas',
                 'GO':'multiple_non_renewable',
                 'LF':'oil',
                 'LV':'oil',
                 'CP/BF':'multiple_non_renewable',
                 'CP/CG':'multiple_non_renewable',
                 'FA/BF':'multiple_non_renewable',
                 'NG/BF':'gas',
                 'NG':'gas',
                 'NU':'uranium',
                 'WR':'waste',
                 'WA':'hydro',
                 'WI':'wind',
                 'WP':'biomass'}
data_BE["Fuel"].replace(dict_fuels_BE, inplace=True)
data_BE["Fuel"].unique()

# Check if all fuels have been translated
for fuel in data_BE["Fuel"].unique():
    if (not fuel in dict_fuels_BE.values()) & (str(fuel) != "nan"):
        logger.error("Untranslated fuel type: " + str(fuel))

# Delete unwanted fuels (wind) in column "fuel"
data_BE = data_BE[data_BE.Fuel != 'wind']

## Generate information concerning Technology

Generation of entries for the column "Technology" according to information given in the columns "Fuel".

In [ ]:
# Generate entries in column "technology" according to fuel "hydro"
data_BE['Technology'][data_BE['Fuel'] == 'hydro'] = 'hydro_plant'

data_BE.head()

# Adjustment of DUTCH data

## Import and merge data

In [ ]:
# Check if input works
data_NL_Q12015 = importdata('NL','Tennet_Q12015')
data_NL_Q22015 = importdata('NL','Tennet_Q22015')
data_NL_Q32015 = importdata('NL','Tennet_Q32015')
data_NL_Q42015 = importdata('NL','Tennet_Q42015')

# Append the lists to one list
data_NL_Q1_Q22015 = data_NL_Q12015.append(data_NL_Q22015)
data_NL_Q1_Q32015 = data_NL_Q1_Q22015.append(data_NL_Q32015)
data_NL_Q1_Q42015 = data_NL_Q1_Q32015.append(data_NL_Q42015)

# Rename the appended list
data_NL = data_NL_Q1_Q42015

## Translate and adjust columns 

Overall adjustment of all columns within the dataframe. Translation, addition, deletion, merger, sorting of columns as well as adjustment of the column entries' types.

In [ ]:
# Add needed columns without data
data_NL['Technology'] = 'UNKNOWN'
data_NL['Type'] = 'NaN'
data_NL['Comment'] = 'NaN'
data_NL['CHP'] = 'NaN'
data_NL['Type'] = 'NaN'
data_NL['Street'] = 'NaN'

# Merge columns "street" and "Number" to one column called "Street"
data_NL['Street'] = data_NL[['street','Number']].apply(lambda x : '{} {}'.format(x[0],x[1]), axis=1)

# Drop columns not needed anymore
colsToDrop = ['Location', 'Date', 'street', 'Number']
data_NL = data_NL.drop(colsToDrop, axis=1)

# Rename columns
dict_columns_NL = {'Connected body':'Company',
                   'Entity':'Name',
                   'zipcode':'Postcode',
                   'place-name':'City',
                   'Country':'Country',
                   'Fuel':'Fuel',
                   'Capacity':'Capacity',
                   'Technology':'Technology',
                   'Type':'Type',
                   'Comment':'Comment',
                   'CHP':'CHP',
                   'Street':'Street',
                   'Source':'Source'}
data_NL.rename(columns=dict_columns_NL, inplace=True)

# Check if all columns have been renamed
for columnnames in data_NL.columns:
    #print(columnnames)
    if not columnnames in dict_columns_NL.values():
        logger.error("Not renamed column: "+ columnnames)

#Adjust types of entries in all columns
data_NL.Company = data_NL.Company.astype(str)
data_NL.Name = data_NL.Name.astype(str)
data_NL.Street = data_NL.Street.astype(str)
data_NL.Postcode = data_NL.Postcode.astype(str)
data_NL.City = data_NL.City.astype(str)
data_NL.Country = data_NL.Country.astype(str)
data_NL.Capacity = data_NL.Capacity.astype(float)
data_NL.Fuel = data_NL.Fuel.astype(str)
data_NL.Technology = data_NL.Technology.astype(str)
data_NL.Type = data_NL.Type.astype(str)
data_NL.CHP = data_NL.CHP.astype(str)
data_NL.Comment = data_NL.Comment.astype(str)
data_NL.Source = data_NL.Source.astype(str)

## Translate and adjust Fuel types

Overall translation of all fuel types mentioned in the column "Fuel" and subsequent translation check. Generation of entries for the column "Technology" according to information given in the column "Fuel".

In [ ]:
# Rename fuel types according to 
# http://www.tennet.org/english/operational_management/system_data_preparation/Reported_production_capacity/Installed_capacity.aspx
dict_fuels_NL = {'E01':'solar',
                 'E02':'wind',
                 'E03':'hydro',
                 'E04':'biomass',
                 'E05':'coal',
                 'E06':'gas',
                 'E07':'oil',
                 'E08':'uranium',
                 'E09':'UNKNOWN'}
data_NL["Fuel"].replace(dict_fuels_NL, inplace=True)
data_NL["Fuel"].unique()

# Check if all fuels have been translated
for fuel in data_NL["Fuel"].unique():
    if (not fuel in dict_fuels_NL.values()) & (str(fuel) != "nan"):
        logger.error("Not renamed fuel type: " + str(fuel))

data_NL['Technology'][data_NL['Fuel'] == 'uranium'] = 'ST'

## Adjust Capacity 

Adjustment of the capacity entry for the row relating to the power plant named "Rijnmond II".

In [ ]:
# data for power plant 'Rijnmond II' are daily total capacity
data_NL['Capacity_new'] = (data_NL['Capacity']/24).where(data_NL.Name == 'Rijnmond II')
data_NL['Capacity'][data_NL.Name == 'Rijnmond II'] = data_NL['Capacity_new']
data_NL = data_NL.drop(['Capacity_new'], axis=1)

# Filter rows by considering "name" and maximum "capacity
#data_NL = data_NL[data_NL['Name'] != data_NL['Name'].shift(-1)]
data_NL = data_NL.sort_values('Capacity', ascending=False).groupby('Name', as_index=False).first()

# Sort order of columns
columns_sorted_NL = ['Company',
                     'Name',
                     'Street',
                     'Postcode',
                     'City',
                     'Country',
                     'Capacity',
                     'Fuel',
                     'Technology',
                     'Type',
                     'CHP',
                     'Comment',
                     'Source']
data_NL = data_NL.reindex(columns=columns_sorted_NL)

data_NL.head()

# Adjustment of ITALIAN data

## Import data

In [ ]:
# Check if input works
data_IT = importdata('IT','Terna')

## Translate and adjust columns

Overall adjustment of all columns within the dataframe. Translation, addition, deletion, sorting of columns as well as adjustment of the column entries' types.

In [ ]:
# Drop columns not needed anymore
colsToDrop = ['CENSIMP',
              'Codice di Rintracciabilità', 
              'Location', 
              'Voltage connection levels [kV]', 
              'Zona', 
              'Regione','Provincia']
data_IT = data_IT.drop(colsToDrop, axis=1)

# Translate columns
dict_columns_IT = {'Descrizione Impianto':'Name',
                   'TIPOLOGIA':'Fuel',
                   'Comune':'City',
                   'PMAX [MW]':'Capacity',
                   'Country':'Country',
                   'Source':'Source'}
data_IT.rename(columns=dict_columns_IT, inplace=True)

# Check if all columns have been translated
for columnnames in data_IT.columns:
    #print(columnnames)
    if not columnnames in dict_columns_IT.values():
        logger.error("Untranslated column: "+ columnnames)
        
# Add columns with empty data
data_IT['Company', 'Street', 'Postcode', 'CHP','Type','Comment'] = 'NaN'
data_IT['Technology']='UNKNOWN'

# Sort columns
columns_sorted_IT = ['Company',
                     'Name',
                     'Street',
                     'Postcode',
                     'City',
                     'Country',
                     'Capacity',
                     'Fuel',
                     'Technology',
                     'Type',
                     'CHP',
                     'Comment',
                     'Source']
data_IT = data_IT.reindex(columns=columns_sorted_IT)

#Adjust types of entries in all columns
data_IT.Company = data_IT.Company.astype(str)
data_IT.Name = data_IT.Name.astype(str)
data_IT.Street = data_IT.Street.astype(str)
data_IT.Postcode = data_IT.Postcode.astype(str)
data_IT.City = data_IT.City.astype(str)
data_IT.Country = data_IT.Country.astype(str)
data_IT.Capacity = data_IT.Capacity.astype(float)
data_IT.Fuel = data_IT.Fuel.astype(str)
data_IT.Technology = data_IT.Technology.astype(str)
data_IT.Type = data_IT.Type.astype(str)
data_IT.CHP = data_IT.CHP.astype(str)
data_IT.Comment = data_IT.Comment.astype(str)
data_IT.Source = data_IT.Source.astype(str)

## Translate and adjust Fuel types

Overall translation of all fuel types mentioned in the column "Fuel" and subsequent translation check. Deletion of rows containing "wind" and "geothermal_power"as fuel type.

In [ ]:
# Translate fuel types
dict_fuels_IT = {'GEOTERMICO':'geothermal_power',
                 'TERMOELETTRICO': 'conventional',
                 'IDROELETTRICO':'hydro',
                 'EOLICO':'wind'}
data_IT["Fuel"].replace(dict_fuels_IT, inplace=True)
data_IT["Fuel"].unique()

# Check if all fuels have been translated
for fuel in data_IT["Fuel"].unique():
    if (not fuel in dict_fuels_IT.values()) & (str(fuel) != "nan"):
        logger.error("Untranslated fuel type: " + str(fuel)) 

# Generate technology entries according to fuels
data_IT['Technology'][data_IT['Fuel'] == 'hydro'] = 'hydro_plant'

# Delete unwanted fuels (wind & geothermal power) in column "fuel"
data_IT = data_IT[data_IT.Fuel != 'wind']
data_IT = data_IT[data_IT.Fuel != 'geothermal_power']

data_IT.head()

# Adjustment of FRENCH data

## Import data

In [ ]:
# Check if input works
data_FR = importdata('FR','RTE')

## Translate and adjust Columns

Overall adjustment of all columns within the dataframe. Translation, addition, deletion, sorting of columns as well as adjustment of the column entries' types.

In [ ]:
# Drop columns not needed anymore
colsToDrop = ['Niveau de tension de connexion (KVT)', 
              'Date de création', 
              'Date de suppression', 
              'Localisation']
data_FR = data_FR.drop(colsToDrop, axis=1)

# Translate columns
dict_columns_FR = {'Type':'Fuel',
                   'Nom de la centrale de production':'Name',
                   'Capacité de production Installée (MW)':'Capacity',
                   'Country':'Country',
                   'Source':'Source'}
data_FR.rename(columns=dict_columns_FR, inplace=True)

# Check if all columns have been translated
for columnnames in data_FR.columns:
    #print(columnnames)
    if not columnnames in dict_columns_FR.values():
        logger.error("Untranslated column: "+ columnnames)

# Add columns with empty data
data_FR['Company', 'Street', 'Postcode', 'City','Technology','CHP', 'Type', 'Comment'] = 'NaN'

# Sort columns
columns_sorted_FR = ['Company',
                     'Name',
                     'Street',
                     'Postcode',
                     'City',
                     'Country',
                     'Capacity',
                     'Fuel',
                     'Technology',
                     'Type',
                     'CHP',
                     'Comment',
                     'Source']
data_FR = data_FR.reindex(columns=columns_sorted_FR)

# Delete unwanted row by referring to column "Name"
data_FR = data_FR.dropna(subset=['Name'])

#Adjust types of entries in all columns
data_FR.Company = data_FR.Company.astype(str)
data_FR.Name = data_FR.Name.astype(str)
data_FR.Street = data_FR.Street.astype(str)
data_FR.Postcode = data_FR.Postcode.astype(str)
data_FR.City = data_FR.City.astype(str)
data_FR.Country = data_FR.Country.astype(str)
data_FR.Capacity = data_FR.Capacity.astype(float)
data_FR.Fuel = data_FR.Fuel.astype(str)
data_FR.Technology = data_FR.Technology.astype(str)
data_FR.Type = data_FR.Type.astype(str)
data_FR.CHP = data_FR.CHP.astype(str)
data_FR.Comment = data_FR.Comment.astype(str)
data_FR.Source = data_FR.Source.astype(str)

## Translate and adjust Fuel types

Overall translation of all fuel types mentioned in the column "Fuel" and subsequent translation check.

In [ ]:
# Translate fuel types
dict_fuels_FR = {'Autre':'UNKNOWN',
                 'Charbon':'coal',
                 'Fioul': 'oil',
                 'Gaz':'gas',
                 'Hydraulique STEP':'pumped_storage',
                 '''Hydraulique fil de l'eau / éclusée''':'run_of_river',
                 'Hydraulique lacs':'reservoir',
                 'Marin':'hydro_tidal',
                 'Nucléaire':'uranium'}
data_FR["Fuel"].replace(dict_fuels_FR, inplace=True)
data_FR["Fuel"].unique()

# Check if all fuels have been translated
for fuel in data_FR["Fuel"].unique():
    if (not fuel in dict_fuels_FR.values()) & (str(fuel) != "nan"):
        logger.error("Untranslated fuel type: " + str(fuel)) 

## Generate Technology types

Generation of entries for the column "Technology" according to information given in the column "Fuel".

In [ ]:
# Generate technology entries according to fuels
data_FR['Technology'][data_FR['Fuel'] == 'pumped_storage'] = 'PSP'
data_FR['Technology'][data_FR['Fuel'] == 'run_of_river'] = 'ROR'
data_FR['Technology'][data_FR['Fuel'] == 'reservoir'] = 'RES'
data_FR['Technology'][data_FR['Fuel'] == 'uranium'] = 'ST'

dict_technologies_FR = {' ':'UNKNOWN'}
data_FR["Technology"].replace(dict_technologies_FR, inplace=True)
data_FR["Technology"].unique()

data_FR.head()

# Adjustment of FINNISH data

## Import data

In [ ]:
# Check if input works
data_FI = importdata('FI','EnergyAuthority')

## Translate and adjust columns

Overall adjustment of all columns within the dataframe. Translation, addition, deletion, sorting of columns as well as adjustment of the column entries' types.

In [ ]:
# Add columns with empty data
data_FI['CHP'] = 'NaN'
data_FI['Comment'] = 'NaN'
data_FI['Technology'] = 'NaN'

# Generate entries in column "CHP"
data_FI['CHP'] = 'No'
data_FI['CHP'][data_FI['Combined Heat and Power Production, Industry,Maximum, Total, MW'] > 0] = 'Yes'
data_FI['CHP'][data_FI['Combined Heat and Power Production, District Heating, Total, MW'] > 0] = 'Yes'

# Drop columns not needed anymore
colsToDrop = ['Business ID', 
              'Location', 
              'Separate power production, Maximum, Hour, MW', 
              'Separate power production, Decomissioned, Hour, MW',
              'Combined Heat and Power Production, Industry,Maximum, Total, MW',
              'Combined Heat and Power Production, Industry,Hour, Total, MW', 
              'Combined Heat and Power Production, Industry, Decomissioned, Total, MW',
              'Combined Heat and Power Production, District Heating, Total, MW',
              'Combined Heat and Power Production, District Heating, Hour, MW', 
              'Combined Heat and Power Production, District Heating, Decomissioned, Total, MW',
              'Separate power production, Maximum, Total, MW',
              'Hour, total, MW',
              'Decomissioned, Total, MW',
              'Standby fuel ',
              'Standby fuel']
data_FI = data_FI.drop(colsToDrop, axis=1)

# Rename columns
dict_columns_FI = {'Name':'Name',
                   'Company':'Company',
                   'Type':'Type',
                   'CHP':'CHP',
                   'Comment':'Comment',
                   'Technology':'Technology',
                   'Address':'Street',
                   'Town':'City',
                   'Postal code':'Postcode',
                   'Maximum, total, MW':'Capacity',
                   'Main fuel':'Fuel',
                   'Country':'Country',
                   'Source':'Source'}
data_FI.rename(columns=dict_columns_FI, inplace=True)

# Check if all columns have been renamed
for columnnames in data_FI.columns:
    #print(columnnames)
    if not columnnames in dict_columns_FI.values():
        logger.error("Not renamed column: "+ columnnames)
    
# Sort columns
columns_sorted_FI = ['Company',
                     'Name',
                     'Street',
                     'Postcode',
                     'City',
                     'Country',
                     'Capacity',
                     'Fuel',
                     'Technology',
                     'Type',
                     'CHP',
                     'Comment',
                     'Source']
data_FI = data_FI.reindex(columns=columns_sorted_FI)

#Adjust types of entries in all columns
data_FI.Company = data_FI.Company.astype(str)
data_FI.Name = data_FI.Name.astype(str)
data_FI.Street = data_FI.Street.astype(str)
data_FI.Postcode = data_FI.Postcode.astype(str)
data_FI.City = data_FI.City.astype(str)
data_FI.Country = data_FI.Country.astype(str)
data_FI.Capacity = data_FI.Capacity.astype(float)
data_FI.Fuel = data_FI.Fuel.astype(str)
data_FI.Technology = data_FI.Technology.astype(str)
data_FI.Type = data_FI.Type.astype(str)
data_FI.CHP = data_FI.CHP.astype(str)
data_FI.Comment = data_FI.Comment.astype(str)
data_FI.Source = data_FI.Source.astype(str)

## Translate and adjust Fuel types

Overall translation of all fuel types mentioned in the column "Fuel" and subsequent translation check. Generation of entries for the column "Fuel" according to information given in the column "Type".

In [ ]:
# Rename fuel types
dict_fuels_FI = {'Biogas':'biogas',
                 'Black liquor and concentrated liquors': 'UNKNOWN',
                 'Blast furnace gas':'gas',
                 'By-products from wood processing industry':'biomass',
                 'Exothermic heat from industry':'UNKNOWN',
                 'Forest fuelwood':'biomass',
                 'Gasified waste':'waste',
                 'Hard coal and anthracite':'coal',
                 'Heavy distillates':'oil',
                 'Industrial wood residues':'biomass',
                 'Light distillates':'oil',
                 'Medium heavy distillates':'oil',
                 'Mixed fuels':'multiple_non_renewable',
                 'Natural gas':'gas',
                 'Nuclear energy':'uranium',
                 'Other by-products and wastes used as fuel':'UNKNOWN',
                 'Other non-specified energy sources':'UNKNOWN',
                 'Peat':'biomass',
                 '':'UNKNOWN'}
data_FI["Fuel"].replace(dict_fuels_FI, inplace=True)
data_FI["Fuel"].unique()

# Check if all fuels have been translated
for fuel in data_FI["Fuel"].unique():
    if (not fuel in dict_fuels_FI.values()) & (str(fuel) != "nan"):
        logger.error("Not renamed fuel type: " + str(fuel)) 

# Generate entries in column "fuel" for hydro and wind stations according to column "type"
data_FI['Fuel'][data_FI['Type'] == 'Hydro power'] = 'hydro'
data_FI['Fuel'][data_FI['Type'] == 'Wind power'] = 'wind'

## Generate Technology types 

Generation of entries for the column "Technology" according to information given in the columns "Type" and "Fuel". Deletion of rows containing "wind" as fuel type.

In [ ]:
# Generate entries in column "technology" according to column "type"
data_FI['Technology'][data_FI['Type'] == 'District heating CHP'] = ''
data_FI['Technology'][data_FI['Type'] == 'Industry CHP'] = ''
data_FI['Technology'][data_FI['Type'] == 'Separate electricity production'] = ''

# Generate entries in column "technology" according to column "fuel"
data_FI['Technology'][data_FI['Fuel'] == 'hydro'] = 'hydro_plant'
data_FI['Technology'][data_FI['Fuel'] == 'uranium'] = 'ST'
data_FI['Technology'][data_FI['Fuel'] == 'wind'] = 'wind_turbine'

# Delete unwanted fuel (wind) in column "fuel"
data_FI = data_FI[data_FI.Fuel != 'wind']

dict_technologies_FI = {' ':'UNKNOWN'}
data_FI["Technology"].replace(dict_technologies_FI, inplace=True)
data_FI["Technology"].unique()

## Translate and adjust Types

Overall translation of all types mentioned in the column "Type" and subsequent translation check.

In [ ]:
# Rename types
dict_types_FI = {'District heating CHP':'CHP',
                 'Hydro power': '',
                 'Industry CHP':'IPP',
                 'Nuclear energy':'',
                 'Separate electricity production':'',
                 'Wind power':''}
data_FI["Type"].replace(dict_types_FI, inplace=True)
data_FI["Type"].unique()

# Check if all types have been translated
for fuel in data_FI["Type"].unique():
    if (not fuel in dict_types_FI.values()) & (str(fuel) != "nan"):
        logger.error("Not renamed type: " + str(fuel)) 

data_FI.head()

# Adjustment of POLISH Data

## Import data

In [ ]:
# Check if input works
data_PL = importdata('PL','GPI')

## Translate and adjust Columns

Overall adjustment of all columns within the dataframe. Translation, completion, addition, deletion, sorting of columns as well as adjustment of the column entries' types.

In [ ]:
# Rename first column
data_PL.columns.values[0] = 'Company'

# Add columns with empty data
data_PL['Street'] = 'NaN'
data_PL['Postcode'] = 'NaN'
data_PL['City'] = 'NaN'
data_PL['Technology'] = 'UNKNOWN'
data_PL['Type'] = 'NaN'
data_PL['CHP'] = 'NaN'

# Rename columns
dict_columns_PL = {'Company':'Company',
                   'Generating unit name':'Name',
                   'Street':'Street',
                   'Postcode':'Postcode',
                   'City':'City',
                   'Technology':'Technology',
                   'Type':'Type',
                   'CHP':'CHP',
                   'Comments':'Comment',
                   'Available capacity [MW]':'Capacity',
                   'Basic fuel':'Fuel',
                   'Country':'Country',
                   'Source':'Source'}
data_PL.rename(columns=dict_columns_PL, inplace=True)

# Check if all columns have been renamed
for columnnames in data_PL.columns:
    #print(columnnames)
    if not columnnames in dict_columns_PL.values():
        logger.error("Not renamed column: "+ columnnames)

# Fill columns "fuel" and "company" with the respective entries
cols = ['Fuel', 'Company']
data_PL[cols] = data_PL[cols].ffill()

# Delete empty and therefore unwanted rows by referring to column "Generating unit code"
data_PL = data_PL.dropna(subset=['Generating unit code'])

# Drop columns not needed anymore
colsToDrop = ['Generating unit code','Voltage [kv]']
data_PL = data_PL.drop(colsToDrop, axis=1)

# Sort columns
columns_sorted_PL = ['Company',
                     'Name',
                     'Street',
                     'Postcode',
                     'City',
                     'Country',
                     'Capacity',
                     'Fuel',
                     'Technology',
                     'Type',
                     'CHP',
                     'Comment',
                     'Source']
data_PL = data_PL.reindex(columns=columns_sorted_PL)

#Adjust types of entries in all columns
data_PL.Company = data_PL.Company.astype(str)
data_PL.Name = data_PL.Name.astype(str)
data_PL.Street = data_PL.Street.astype(str)
data_PL.Postcode = data_PL.Postcode.astype(str)
data_PL.City = data_PL.City.astype(str)
data_PL.Country = data_PL.Country.astype(str)
data_PL.Capacity = data_PL.Capacity.astype(float)
data_PL.Fuel = data_PL.Fuel.astype(str)
data_PL.Technology = data_PL.Technology.astype(str)
data_PL.Type = data_PL.Type.astype(str)
data_PL.CHP = data_PL.CHP.astype(str)
data_PL.Comment = data_PL.Comment.astype(str)
data_PL.Source = data_PL.Source.astype(str)

## Translate and adjust Fuel types

Overall translation of all fuel types mentioned in the column "Fuel" and subsequent translation check.

In [ ]:
# Rename fuel types
dict_fuels_PL = {'Brown coal':'lignite',
                 'Black coal':'coal',
                 'Water':'hydro'}
data_PL["Fuel"].replace(dict_fuels_PL, inplace=True)
data_PL["Fuel"].unique()

# Check if all fuels have been translated
for fuel in data_PL["Fuel"].unique():
    if (not fuel in dict_fuels_PL.values()) & (str(fuel) != "nan"):
        logger.error("Not renamed fuel type: " + str(fuel))

## Generate Technology types

Generation of entries for the column "Technology" according to information given in the column "Fuel".

In [ ]:
# Generate entries in column "technology" according to fuel "hydro"
data_PL['Technology'][data_PL['Fuel'] == 'hydro'] = 'hydro_plant'

data_PL.head()

# Adjustment of SPANISH data

## Import data

In [ ]:
# Check if input works
data_ES = importdata('ES','SEDE')

## Translate and adjust Columns

TODO: Decision, which of both columns "Net_Capacity" and "Capacity" will be dropped, has to be taken.

Overall adjustment of all columns within the dataframe. Translation, addition, deletion, sorting of columns as well as adjustment of the column entries' types. Adjustment of the entries' units from kW to MW in the columns "Net_Capacity" and "Capacity".

In [ ]:
# Drop columns not needed anymore
colsToDrop = ['N. Orden',
              'Tipo Regimen',
              'Autonomia',
              'Provincia',
              'F. Puesta En Servicio',
              'F. Alta',
              'F. Baja',
              'F. Alta Provicional',
              'Alta Registro']
data_ES = data_ES.drop(colsToDrop, axis=1)

# Add columns with empty data
data_ES['Street'] = 'NaN'
data_ES['Postcode'] = 'NaN'
data_ES['Type'] = 'NaN'
data_ES['CHP'] = 'NaN'

# Rename columns
dict_columns_ES = {'Titular':'Company',
                   'Nombre de la instalacion':'Name',
                   'Street':'Street',
                   'Postcode':'Postcode',
                   'Municipio':'City',
                   'Tecnologia':'Technology',
                   'Type':'Type',
                   'CHP':'CHP',
                   'Comment':'Comment',
                   'Potencia Neta':'Net_Capacity',
                   'Potencia Bruta':'Capacity',
                   'Combustible':'Fuel',
                   'Country':'Country',
                   'Source':'Source'}
data_ES.rename(columns=dict_columns_ES, inplace=True)

# Check if all columns have been renamed
for columnnames in data_ES.columns:
    #print(columnnames)
    if not columnnames in dict_columns_ES.values():
        logger.error("Not renamed column: "+ columnnames)
        
# Sort columns
columns_sorted_ES = ['Company',
                     'Name',
                     'Street',
                     'Postcode',
                     'City',
                     'Country',
                     'Net_Capacity',
                     'Capacity',
                     'Fuel',
                     'Technology',
                     'Type',
                     'CHP',
                     'Comment',
                     'Source']
data_ES = data_ES.reindex(columns=columns_sorted_ES)


# Change unit of column 'net_capacity' from kW to MW
data_ES.Net_Capacity = data_ES.Net_Capacity.astype(float)
data_ES['Net_Capacity'] = (data_ES['Net_Capacity']/1000)

# Change unit of column 'capacity' from kW to MW
data_ES.Capacity = data_ES.Capacity.astype(float)
data_ES['Capacity'] = (data_ES['Capacity']/1000)

#Adjust types of entries in all columns
data_ES.Company = data_ES.Company.astype(str)
data_ES.Name = data_ES.Name.astype(str)
data_ES.Street = data_ES.Street.astype(str)
data_ES.Postcode = data_ES.Postcode.astype(str)
data_ES.City = data_ES.City.astype(str)
data_ES.Country = data_ES.Country.astype(str)
data_ES.Fuel = data_ES.Fuel.astype(str)
data_ES.Technology = data_ES.Technology.astype(str)
data_ES.Type = data_ES.Type.astype(str)
data_ES.CHP = data_ES.CHP.astype(str)
data_ES.Comment = data_ES.Comment.astype(str)
data_ES.Source = data_ES.Source.astype(str)

## Translate and adjust Fuel types 

Overall translation of all fuel types mentioned in the column "Fuel" and subsequent translation check.

In [ ]:
dict_fuels_ES = {'Biocombustibles liquidos':'biomass',
                 'Biogas': 'biogas',
                 'Biogas de digestion':'biogas',
                 'Biogas de vertedero':'biogas',
                 'Biomasa industrial agricola':'biomass',
                 'Biomasa industrial forestal':'biomass',
                 'Biomasa primaria':'biomass',
                 'Calor residual':'UNKNOWN',
                 'Carbon':'coal',
                 'CARBON IMPORTADO':'coal',
                 'Cultivos energeticos agricolas o forestales':'biomass',
                 'DIESEL':'oil',
                 'Energias residuales':'UNKNOWN',
                 'Fuel':'oil',
                 'FUEL-OIL 0,3':'oil',
                 'FUELOLEO':'oil',
                 'GAS DE REFINERIA':'gas',                 
                 'Gas natural':'gas',
                 'GAS NATURAL':'gas',
                 'Gas residual':'gas',
                 'Gasoleo':'gas',
                 'GASOLEO':'gas',
                 'HULLA+ANTRACITA':'coal',
                 'Licores negros':'biomass',
                 'LIGNITO NEGRO':'lignite',
                 'LIGNITO PARDO':'lignite',
                 'NUCLEAR':'uranium',
                 'Propano':'gas',
                 'Residuo aprovechamiento forestal o selvicola':'waste',
                 'Residuos':'waste',
                 'Residuos actividad agricolas o jardineria':'waste',
                 'Residuos industriales':'waste',
                 'Residuos solidos urbanos':'waste',
                 'RESIDUOS SOLIDOS URBANOS':'waste',
                ' ':'UNKNOWN'}
data_ES["Fuel"].replace(dict_fuels_ES, inplace=True)
data_ES["Fuel"].unique()

# Check if all fuels have been translated
for fuel in data_ES["Fuel"].unique():
    if (not fuel in dict_fuels_ES.values()) & (str(fuel) != "nan"):
        logger.error("Not renamed fuel type: " + str(fuel))

## Translate and adjust Technology types

TODO: 
- Open question on meaning of technology "cogeneratcion" (CCGT or CHP?)
- Exlcude power plants at Baleares, Canarias, Melilla, and Ceuta

Overall translation of all technology types mentioned in the column "Technology" and subsequent translation check.

In [ ]:
dict_technologies_ES = {'Cogeneracion':'CC',
                        'Eolica terrestre': 'wind_turbine',
                        'Fotovoltaica':'solar_plant',
                        'Hidraulica':'hydro_plant',
                        'Hidraulica fluyente':'hydro_plant',
                        'Termonuclear':'ST',
                        'Turbina de gas':'GT',
                       ' ':'UNKNOWN'}
data_ES["Technology"].replace(dict_technologies_ES, inplace=True)
data_ES["Technology"].unique()

# Check if all technologies have been translated
for fuel in data_ES["Technology"].unique():
    if (not fuel in dict_technologies_ES.values()) & (str(fuel) != "nan"):
        logger.error("Untranslated technology: " + str(fuel))

## Generate and adjust Fuel types

Generation of entries for the column "Fuel" according to information given in the column "Technology". Deletion of rows containing "wind" and "solar" as fuel type.

In [ ]:
# Generate entries in column "fuel" for hydro, wind and solar stations according to column "technology"
data_ES['Fuel'][data_ES['Technology'] == 'hydro_plant'] = 'hydro'
data_ES['Fuel'][data_ES['Technology'] == 'solar_plant'] = 'solar'
data_ES['Fuel'][data_ES['Technology'] == 'wind_turbine'] = 'wind'

# Delete unwanted fuels
data_ES = data_ES[data_ES.Fuel != 'wind']
data_ES = data_ES[data_ES.Fuel != 'solar']

# Generate entries in column "CHP" according to column "Technology"
#data_ES['CHP'][data_ES['Technology'] == 'CC'] = 'Yes'

# Generate entries in column "Type" according to column "Technology"
#data_ES['Type'][data_ES['Technology'] == 'CC'] = 'CHP'

data_ES.head()

# Adjustment of BRITISH data

## Import data

In [ ]:
# Check if input works
data_UK = importdata('UK','GOV')

# Translate and adjust Columns

Overall adjustment of all columns within the dataframe. Translation, addition, deletion, sorting of columns as well as adjustment of the column entries' types.

In [ ]:
# Rename sixth column
data_UK.columns.values[5] = 'Location'

# Drop rows without station names, so that the footnotes at the end of the list are deleted
data_UK = data_UK.dropna(subset=['Station Name'])

# Drop columns not needed anymore
colsToDrop = ['Year of commission or year generation began', 'Footnotes']
data_UK = data_UK.drop(colsToDrop, axis=1)

# Add needed columns without data
data_UK['Street'] = 'NaN'
data_UK['Postcode'] = 'NaN'
data_UK['City'] = 'NaN'
data_UK['Technology'] = 'UNKNOWN'
data_UK['CHP'] = 'NaN'
data_UK['Type'] = 'NaN'
data_UK['Comment'] = 'NaN'

# Rename columns
dict_columns_UK = {'Company Name':'Company',
                   'Station Name':'Name',
                   'Installed Capacity (MW)':'Capacity',
                   'Street':'Street',
                   'Postcode':'Postcode',
                   'City':'City',
                   'Country':'Country',
                   'Location':'Location',
                   'Fuel':'Fuel',
                   'Technology':'Technology',
                   'Type':'Type',
                   'CHP':'CHP',
                   'Comment':'Comment',
                   'Source':'Source'}
data_UK.rename(columns=dict_columns_UK, inplace=True)

# Check if all columns have been renamed
for columnnames in data_UK.columns:
    #print(columnnames)
    if not columnnames in dict_columns_UK.values():
        logger.error("Not renamed column: "+ columnnames)

# Sort columns
columns_sorted_UK = ['Company',
                     'Name',
                     'Street',
                     'Postcode',
                     'City',
                     'Country',
                     'Location',
                     'Capacity',
                     'Fuel',
                     'Technology',
                     'Type',
                     'CHP',
                     'Comment',
                     'Source']
data_UK = data_UK.reindex(columns=columns_sorted_UK)

# Merge columns "Country" and "Location" to one column called "Country"
data_UK['Country'] = data_UK[['Country','Location']].apply(lambda x : '{} - {}'.format(x[0],x[1]), axis=1)

# Drop column "Location" after merger
colsToDrop = ['Location']
data_UK = data_UK.drop(colsToDrop, axis=1)

#Adjust types of entries in all columns
data_UK.Company = data_UK.Company.astype(str)
data_UK.Name = data_UK.Name.astype(str)
data_UK.Street = data_UK.Street.astype(str)
data_UK.Postcode = data_UK.Postcode.astype(str)
data_UK.City = data_UK.City.astype(str)
data_UK.Country = data_UK.Country.astype(str)
data_UK.Capacity = data_UK.Capacity.astype(float)
data_UK.Fuel = data_UK.Fuel.astype(str)
data_UK.Technology = data_UK.Technology.astype(str)
data_UK.Type = data_UK.Type.astype(str)
data_UK.CHP = data_UK.CHP.astype(str)
data_UK.Comment = data_UK.Comment.astype(str)
data_UK.Source = data_UK.Source.astype(str)

## Generate and adjust Technology types 

Generation of entries for the column "Technology" according to information given in the column "Fuel".

In [ ]:
# Generate entries in column "technology" according to hydro fuels
data_UK['Technology'][data_UK['Fuel'] == 'Hydro'] = 'hydro_plant'
data_UK['Technology'][data_UK['Fuel'] == 'Hydro / pumped storage'] = 'PSP'
data_UK['Technology'][data_UK['Fuel'] == 'Pumped storage'] = 'PSP'
data_UK['Technology'][data_UK['Fuel'] == 'Wind'] = 'wind_turbine'
data_UK['Technology'][data_UK['Fuel'] == 'Wind (offshore)'] = 'wind_turbine'
data_UK['Technology'][data_UK['Fuel'] == 'Nuclear'] = 'ST'

## Translate and adjust Fuel types

Overall translation of all fuel types mentioned in the column "Fuel" and subsequent translation check. Deletion of rows containing "wind" as fuel type.

In [ ]:
dict_fuels_UK = {'Biomass':'biomass',
                 'CCGT': 'UNKNOWN',
                 'Coal':'coal',
                 'Coal / biomass':'multiple_non_renewable',
                 'Coal / biomass / gas / waste derived fuel':'multiple_non_renewable',
                 'Coal / oil':'multiple_non_renewable',
                 'Diesel':'oil',
                 'Gas':'gas',
                 'Gas / oil':'multiple_non_renewable',
                 'Gas oil':'multiple_non_renewable',
                 'Gas oil / kerosene':'multiple_non_renewable',
                 'Hydro':'hydro',
                 'Hydro / pumped storage':'hydro',
                 'Light oil':'oil',
                 'Meat & bone meal':'UNKNOWN',
                 'Nuclear':'uranium',
                 'OCGT':'UNKNOWN',                 
                 'Oil':'oil',
                 'Pumped storage':'hydro',
                 'Straw':'biomass',
                 'Waste':'waste',
                 'Wind':'wind',
                 'Wind (offshore)':'wind'}
data_UK["Fuel"].replace(dict_fuels_UK, inplace=True)
data_UK["Fuel"].unique()

# Check if all fuels have been translated
for fuel in data_UK["Fuel"].unique():
    if (not fuel in dict_fuels_UK.values()) & (str(fuel) != "nan"):
        logger.error("Not renamed fuel type: " + str(fuel))
        
# Delete unwanted fuels
data_UK = data_UK[data_UK.Fuel != 'wind']

# Create output-files

In [ ]:
data_ES.to_csv('data_processed/data_ES.csv')
data_PL.to_csv('data_processed/data_PL.csv')
data_NL.to_csv('data_processed/data_NL.csv')
data_FR.to_csv('data_processed/data_FR.csv')
data_BE.to_csv('data_processed/data_BE.csv')
data_FI.to_csv('data_processed/data_FI.csv')
data_IT.to_csv('data_processed/data_IT.csv')
data_UK.to_csv('data_processed/data_UK.csv')

# Documenting the data package (meta data)

We document the data packages meta data in the specific format JSON as proposed by the Open Knowledge Foundation. See the Frictionless Data project by OKFN (http://data.okfn.org/) and the Data Package specifications (http://dataprotocols.org/data-packages/) for more details.

In order to keep the notebook more readable, we first formulate the metadata in the human-readable YAML format using a multi-line string. We then parse the string into a Python dictionary and save that to disk as a JSON file.

In [ ]:
# Here we define meta data of the resulting data package.
# The meta data follows the specification at:
# http://dataprotocols.org/data-packages/

metadata = """

name: opsd-power-plants-europe
title: Power plants of European countries
description: 
version: "2016-04-22"
keywords: [power plants,europe]
opsd-jupyter-notebook-url: "https://github.com/Open-Power-System-Data/datapackage_power_plants_europe/blob/master/main.ipynb"
geographical-scope: Europe
opsd-changes-to-last-version: xxx

resources:
    - path: xxx.csv
      format: csv
      mediatype: text/csv
      schema:    
        fields:
            - name:
              description:
              type: 
    - path: xxx.xlsx
      format: xlsx
      mediatype: application/vnd.openxmlformats-officedocument.spreadsheetml.sheet
              
licenses:
    - url: http://example.com/license/url/here
      name: License Name Here
      version: 1.0
      id: license-id-from-open

sources:
    - name: 
      web: 
maintainers:
    - name: Friedrich Kunz
      email: fkunz@diw.de
      web: http://open-power-system-data.org/

openpowersystemdata-enable-listing: True  


"""

metadata = yaml.load(metadata)

datapackage_json = json.dumps(metadata, indent=4, separators=(',', ': '))

# Write results to file

In [ ]:
output_path = 'data_final/'

#Write the result to file
#data.to_csv(output_path+'power_plants_europe.csv', encoding='utf-8')

#Write the results to excel file
#data.to_excel(output_path+'power_plants_europe.xlsx', sheet_name='output')

#Write the results to sql database
#data.to_sql(output_path+'power_plants_europe', sqlite3.connect(output_path+'power_plants_europe.sqlite'), if_exists="replace") 

#Write the information of the metadata
with open(os.path.join(output_path, 'datapackage.json'), 'w') as f:
    f.write(datapackage_json)

#Set this string to this notebook's filename!    
nb_filename = 'main.ipynb'

# Save a copy of the notebook to markdown, to serve as the package README file
subprocess.call(['ipython', 'nbconvert', '--to', 'markdown', nb_filename])
path_readme = os.path.join(output_path, 'README.md')
try:
    os.remove(path_readme)
except Exception:
    pass
os.rename(nb_filename.replace('.ipynb', '.md'), path_readme)    

# STEINBRUCH [not relevant]

In [ ]:
now = datetime.datetime.now()
datestring = ""
datestring = str(now.year)+"-"+str(now.month)+"-"+str(now.day)
data=pd.DataFrame()
data_import=pd.DataFrame()

In [ ]:
# RTE
param = conf['FR']['RTE']
filepath = "data_downloaded/"+datestring+"-"+str(param['filename'])+"."+param['filetype']
if param['filetype'] == 'csv':
    data_import = pd.read_csv(filepath,
                              sep=param['sep'],
                              skiprows=param['skiprows'],
                              decimal=param['decimal'],   
                              encoding=param['encoding'])

data_import.head()

In [ ]:
#### NOTE: May not work for several datauploads with differing table structure

now = datetime.datetime.now()
datestring = ""
datestring = str(now.year)+"-"+str(now.month)+"-"+str(now.day)
data=pd.DataFrame()
data_import=pd.DataFrame()

for country, tso in conf.items():
    for tso, param in tso.items():
        filepath = "data_downloaded/"+datestring+"-"+str(param['filename'])+"."+param['filetype']
        if param['filetype'] == 'csv':
            data_import = pd.read_csv(filepath, 
                            sep=param['sep'],
                            skiprows=param['skiprows'],
                            decimal=param['decimal'],   
                            encoding=param['encoding'])
        data = data.append(data_import)
        
data.head(500)